In [14]:
SOURCE = '../data/caterpillar/caterpillar-tube-pricing/competition_data/';

In [12]:
from glob import glob;
import os;
import pandas as pd;

In [17]:
def search_column(name):
    for path in glob(os.path.join(SOURCE, '*.csv')):
        df = pd.read_csv(path);
        if name in df.columns:
            print(path, df.shape);
            print(df.columns.tolist(), '\n\n');
            
search_column('weight');

../data/caterpillar/caterpillar-tube-pricing/competition_data/comp_threaded.csv (194, 32)
['component_id', 'component_type_id', 'adaptor_angle', 'overall_length', 'hex_size', 'end_form_id_1', 'connection_type_id_1', 'length_1', 'thread_size_1', 'thread_pitch_1', 'nominal_size_1', 'end_form_id_2', 'connection_type_id_2', 'length_2', 'thread_size_2', 'thread_pitch_2', 'nominal_size_2', 'end_form_id_3', 'connection_type_id_3', 'length_3', 'thread_size_3', 'thread_pitch_3', 'nominal_size_3', 'end_form_id_4', 'connection_type_id_4', 'length_4', 'thread_size_4', 'thread_pitch_4', 'nominal_size_4', 'unique_feature', 'orientation', 'weight'] 


../data/caterpillar/caterpillar-tube-pricing/competition_data/comp_adaptor.csv (25, 20)
['component_id', 'component_type_id', 'adaptor_angle', 'overall_length', 'end_form_id_1', 'connection_type_id_1', 'length_1', 'thread_size_1', 'thread_pitch_1', 'nominal_size_1', 'end_form_id_2', 'connection_type_id_2', 'length_2', 'thread_size_2', 'thread_pitch_2', 

In [28]:
df_list = [pd.read_csv(path) for path in glob(os.path.join(SOURCE, 'comp_*.csv'))];

In [29]:
comp = pd.concat(df_list, sort=False);
columns = ['component_id', 'component_type_id', 'orientation', 'unique_feature', 'weight'];
comp = comp[columns];
comp['orientation'] = (comp['orientation'] == 'Yes').astype(int);
comp['unique_feature'] = (comp['unique_feature'] == 'Yes').astype(int);
comp['weight'] = comp['weight'].fillna(comp['weight'].median());

In [30]:
comp.head()

,component_id,component_type_id,orientation,unique_feature,weight
0,C-0007,CP-014,0,0,0.028
1,C-0030,CP-015,1,0,0.914
2,C-0041,CP-014,0,0,0.120
3,C-0043,CP-014,0,0,0.090
4,C-0044,CP-014,0,0,0.182


In [18]:
comp_threaded = pd.read_csv(os.path.join(SOURCE, 'comp_threaded.csv'));
comp_threaded['orientation'].value_counts();

No     121
Yes     73
Name: orientation, dtype: int64

In [19]:
comp_threaded['unique_feature'].value_counts()

No     161
Yes     33
Name: unique_feature, dtype: int64

In [21]:
materials = pd.read_csv(os.path.join(SOURCE, 'bill_of_materials.csv'));
materials.head()

,tube_assembly_id,component_id_1,quantity_1,component_id_2,quantity_2,component_id_3,quantity_3,component_id_4,quantity_4,component_id_5,quantity_5,component_id_6,quantity_6,component_id_7,quantity_7,component_id_8,quantity_8
0,TA-00001,C-1622,2.0,C-1629,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TA-00002,C-1312,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TA-00003,C-1312,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TA-00004,C-1312,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TA-00005,C-1624,1.0,C-1631,1.0,C-1641,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
materials['components_total'] = sum(materials[f'quantity_{n}'].fillna(0) for n in range(1, 9));
materials['components_distinct'] = sum(materials[f'component_id_{n}'].notnull().astype(int) for n in range(1, 9));
materials['orientation'] = 0
materials['unique_feature'] = 0
materials['weight'] = 0

for n in range(1, 9):
    materials = materials.merge(comp, left_on=f'component_id_{n}', right_on='component_id', how='left', suffixes=('', f'_{n}'));

for col in materials:
    if 'orientation' in col or 'unique_feature' in col or 'weight' in col:
        materials[col] = materials[col].fillna(0)
        
materials['orientation'] = sum(materials[f'orientation_{n}'] for n in range(1, 9));
materials['unique_feature'] = sum(materials[f'unique_feature_{n}'] for n in range(1, 9));
materials['weight'] = sum(materials[f'weight_{n}'] for n in range(1, 9));

materials.head()

,tube_assembly_id,component_id_1,quantity_1,component_id_2,quantity_2,component_id_3,quantity_3,component_id_4,quantity_4,component_id_5,...,component_id_7,component_type_id_7,orientation_7,unique_feature_7,weight_7,component_id_8,component_type_id_8,orientation_8,unique_feature_8,weight_8
0,TA-00001,C-1622,2.0,C-1629,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0
1,TA-00002,C-1312,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0
2,TA-00003,C-1312,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0
3,TA-00004,C-1312,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0
4,TA-00005,C-1624,1.0,C-1631,1.0,C-1641,1.0,NaN,NaN,NaN,...,NaN,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0
